In [1]:
import iqs_client
from iqs_client import QueryExecutionApi, QueriesApi

from py2neo import Node, Relationship, Graph, Subgraph

In [2]:
import time
import json
import getpass

In [3]:
configuration = iqs_client.Configuration()
configuration.username='chammard'
configuration.password=getpass.getpass()
configuration.host='http://cae-incquery-uat.jpl.nasa.gov:47700/api'
api_client=iqs_client.ApiClient(configuration)

 ······················


In [4]:
query_api=QueryExecutionApi(api_client=api_client)

In [5]:
graph = Graph(bolt='bolt://localhost:7687/db/data', password="password")

In [6]:
# Empty graph
# graph.run("MATCH (n) DETACH DELETE n")

In [7]:
body = """{{
  "modelCompartment": {{
    "compartmentURI": "twc-index:/workspaces/3d9fe728-a0b3-40e7-9f47-0af95488684e/resources/fd5cf2ca-7357-402c-835d-d066acadf2f5/branches/eb3b9468-78b2-45ba-821b-ade97d9c3570/revisions/6"
  }},
  "queryFQN": "cref.test.{query}"
}}"""

## Nodes

In [8]:
node_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getNamespaces2")))

node_dict = dict()

In [9]:
for match in node_response.to_dict()['matches']:
    uuid = match['arguments'][0]['value']['relativeElementID']
    name = match['arguments'][1]['value']
    element_type = match['arguments'][2]['value']
    if uuid not in node_dict:
        node_dict[uuid] = Node(element_type, uuid=uuid, name=name)
    else:
        node_dict[uuid].add_label(element_type)

In [10]:
no_label_resp = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getNamespacesNoStereotype")))

for match in no_label_resp.to_dict()['matches']:
    uuid = match['arguments'][0]['value']['relativeElementID']
    name = match['arguments'][1]['value']
    node_dict[uuid] = Node("NoAppliedStereotype", uuid=uuid, name=name)

## Edges

In [11]:
edge_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getNamespaceRelations")))

edge_list = []

In [12]:
for match in edge_response.to_dict()['matches']:
    source_uuid = match['arguments'][0]['value']['relativeElementID']
    target_uuid = match['arguments'][1]['value']['relativeElementID']
    try:
        edge_list.append(Relationship(node_dict[source_uuid], "Edge", node_dict[target_uuid], weight=1))
    except:
        pass

### Part Properties

In [13]:
parts_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getPartProperties")))

In [14]:
for match in parts_response.to_dict()['matches']:
    part_uuid = match['arguments'][0]['value']['relativeElementID']
    part_name = match['arguments'][1]['value']
    owner_uuid = match['arguments'][2]['value']['relativeElementID']
    type_uuid = match['arguments'][4]['value']['relativeElementID']

    node_dict[part_uuid] = Node("PartProperty", uuid=part_uuid, name=part_name)
    
    if (type_uuid not in node_dict) or (owner_uuid not in node_dict):
        raise ValueError("Error: Either type or owner is not yet in the graph")
    
    edge_list.append(Relationship(node_dict[owner_uuid], "ATTRIBUTE", node_dict[part_uuid], weight=1))
    edge_list.append(Relationship(node_dict[part_uuid], "TYPED", node_dict[type_uuid], weight=1))

### Reference Properties

In [15]:
reference_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getReferenceProperties")))

In [16]:
for match in reference_response.to_dict()['matches']:
    prop_uuid = match['arguments'][0]['value']['relativeElementID']
    prop_name = match['arguments'][1]['value']
    owner_uuid = match['arguments'][2]['value']['relativeElementID']
    type_uuid = match['arguments'][4]['value']['relativeElementID']

    node_dict[prop_uuid] = Node("PartProperty", uuid=prop_uuid, name=prop_name)
    
    if (type_uuid not in node_dict) or (owner_uuid not in node_dict):
        print(type_uuid, owner_uuid)
        raise ValueError("Error: Either type or owner is not yet in the graph")
    
    edge_list.append(Relationship(node_dict[owner_uuid], "ATTRIBUTE", node_dict[prop_uuid], weight=1))
    edge_list.append(Relationship(node_dict[prop_uuid], "TYPED", node_dict[type_uuid], weight=1))

### Value Properties

In [17]:
value_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getValueProperties")))

WARNING: Libraries of value type are not included in the query.\
Quickfix was to create it manually from the IQS response, but it is not included in the "structure" of the model.

In [18]:
for match in value_response.to_dict()['matches']:
    prop_uuid = match['arguments'][0]['value']['relativeElementID']
    prop_name = match['arguments'][1]['value']
    owner_uuid = match['arguments'][2]['value']['relativeElementID']
    value_type_uuid = match['arguments'][4]['value']['relativeElementID']
    value_type_name = match['arguments'][5]['value']

    node_dict[prop_uuid] = Node("ValueProperty", uuid=prop_uuid, name=prop_name)
    
    if value_type_uuid not in node_dict:
        print(value_type_uuid, value_type_name)
        node_dict[value_type_uuid] = Node("ValueType",  uuid=value_type_uuid, name=value_type_name)
    
    if (value_type_uuid not in node_dict) or (owner_uuid not in node_dict):
        raise ValueError("Error: Either type or owner is not yet in the graph")

    edge_list.append(Relationship(node_dict[owner_uuid], "ATTRIBUTE", node_dict[prop_uuid], weight=1))
    edge_list.append(Relationship(node_dict[prop_uuid], "TYPED", node_dict[value_type_uuid], weight=1))


cb9c4145-c43a-4457-b7e3-597b21cf9ada Integer
410e7cbd-cf5f-4e44-929f-c9cf6ed7f359 Real
55bc3750-0764-40c0-b527-18b101cfbbdb m
33f0a215-9001-4971-b8be-3417fb727403 W
72a37f5d-d1e3-4235-92a5-d7ee568d8a95 s
208ec2fe-d94c-48f7-a479-188137ea8255 Boolean


### Get default values

In [19]:
default_value_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getValuePropertiesDefaultStar2")))

In [20]:
for match in default_value_response.to_dict()['matches']:
    prop_uuid = match['arguments'][0]['value']['relativeElementID']
    prop_name = match['arguments'][1]['value']
    default_value = match['arguments'][2]['value']
    value_type_name = match['arguments'][3]['value']

    node_dict['defaultVal' + prop_uuid] = Node("DefaultValue", default_value=default_value, value_type=value_type_name)
    
    edge_list.append(Relationship(node_dict[prop_uuid], "DEFAULT", node_dict['defaultVal' + prop_uuid], weight=1))

### Requirement text

In [21]:
req_text_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getRequirementText2")))

In [22]:
for match in req_text_response.to_dict()['matches']:
    req_uuid = match['arguments'][0]['value']['relativeElementID']
    req_text = match['arguments'][1]['value']
    
    # Add to the requirement node in node_dict the req_text property
    node_dict[req_uuid]['req_text'] = req_text

In [23]:
with open("test.csv", "a") as myfile:
    for match in req_text_response.to_dict()['matches']:
        req_uuid = match['arguments'][0]['value']['relativeElementID']
        req_text = match['arguments'][1]['value']
        myfile.write(req_uuid + ',' + req_text + '\n')

### Activities

In [24]:
activities_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getActivities")))

In [25]:
for match in activities_response.to_dict()['matches']:
    activity_uuid = match['arguments'][0]['value']['relativeElementID']
    activity_node_uuid = match['arguments'][2]['value']['relativeElementID']
    activity_node_name = match['arguments'][3]['value']
    
    node_dict[activity_uuid].add_label('Activity')
    
    if activity_node_uuid in node_dict:
        node_dict[activity_node_uuid].add_label('ActivityNode')
    else:
        node_dict[activity_node_uuid] = Node('ActivityNode', uuid=activity_node_uuid, name=activity_node_name)
        
    edge_list.append(Relationship(node_dict[activity_uuid], "ACTIVITY_NODE", node_dict[activity_node_uuid]))

### Call Behaviors

In [26]:
behavior_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getBehaviors9")))

In [27]:
for match in behavior_response.to_dict()['matches']:
    call_behavior_uuid = match['arguments'][0]['value']['relativeElementID']
    call_behavior_name = match['arguments'][1]['value']
    behavior_uuid = match['arguments'][2]['value']['relativeElementID']
    
    if call_behavior_uuid in node_dict:
        node_dict[call_behavior_uuid].add_label('CallBehaviorAction')
    else:
        node_dict[call_behavior_uuid] = Node('CallBehaviorAction', uuid=call_behavior_uuid, name=call_behavior_name)
        
    node_dict[behavior_uuid].add_label('Behavior')
    
    edge_list.append(Relationship(node_dict[call_behavior_uuid], "CALLS", node_dict[behavior_uuid]))

### Behaviored Classifiers

In [28]:
behavior_classif_response = query_api.execute_query_on_model_compartment(json.loads(body.format(query="getBehavioredClassifiers")))

In [29]:
for match in behavior_classif_response.to_dict()['matches']:
    classifier_uuid = match['arguments'][0]['value']['relativeElementID']
    behavior_uuid = match['arguments'][2]['value']['relativeElementID']
    
    edge_list.append(Relationship(node_dict[classifier_uuid], "CLASSIFIER_BEHAVIOR", node_dict[behavior_uuid]))

In [30]:
time.time()

1589588667.755264

## Create graph

In [31]:
time1 = time.time()
for node in node_dict.values():
    try:
        graph.create(node)
    except:
        print(node)
for edge in edge_list:
    graph.create(edge)
print('create exec time: ', time.time()-time1)

create exec time:  240.34903383255005
